# Part 3: Geometric Analysis & Visualization

This notebook performs deep geometric analysis of the Delta Observer's latent space and generates the figures from the paper.

## Key Questions

1. **Linear Accessibility:** Can we predict semantic properties (carry count) from latent space using linear models?
2. **Geometric Clustering:** Are points with similar semantics clustered together in space?
3. **The Paradox:** Can information be linearly accessible WITHOUT geometric clustering?

---

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, r2_score
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
import umap
import sys
sys.path.append('..')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("Environment ready for analysis")

## Load Latent Space

Load the 16D latent representations extracted from the trained Delta Observer.

In [ ]:

latent_umap = data['latent_2d']  # 2D UMAP projection
bit_positions = data['bit_positions']  # 0-3 bit positions


# Setup models directory
possible_models_dirs = ['../models', 'models', 'delta-observer/models']
models_dir = None
for p_model in possible_models_dirs:
    if os.path.exists(p_model):
        models_dir = p_model
        break
if not models_dir:
    models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)
print(f'✓ Models directory: {models_dir}')

# Note: 'inputs' not available in this dataset


## Dimensionality Reduction

### PCA: Variance Analysis

PCA reveals how much information is captured in the first few dimensions.

In [ ]:
# PCA analysis
pca = PCA()
latent_pca = pca.fit_transform(latent_space)
explained_var = pca.explained_variance_ratio_

print("PCA Variance Explained:")
print(f"  PC1: {explained_var[0]:.2%}")
print(f"  PC2: {explained_var[1]:.2%}")
print(f"  PC1+PC2: {explained_var[:2].sum():.2%}")
print(f"  Top 3: {explained_var[:3].sum():.2%}")
print(f"  Top 5: {explained_var[:5].sum():.2%}")

# Visualize variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
ax1.bar(range(1, len(explained_var)+1), explained_var, alpha=0.7)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Variance Explained')
ax1.set_title('PCA Scree Plot', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Cumulative variance
cumsum_var = np.cumsum(explained_var)
ax2.plot(range(1, len(cumsum_var)+1), cumsum_var, 'o-', linewidth=2)
ax2.axhline(0.95, color='r', linestyle='--', label='95% threshold')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Variance Explained')
ax2.set_title('Cumulative Variance', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'pca_variance.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ The latent space is relatively low-dimensional:")
print(f"   {np.argmax(cumsum_var > 0.95) + 1} components explain 95% of variance")

### UMAP: Nonlinear Projection

UMAP preserves local structure better than PCA for visualization.

In [ ]:
# UMAP projection
print("Computing UMAP projection...")
reducer = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    metric='euclidean',
    random_state=42
)
latent_umap = reducer.fit_transform(latent_space)
print("✅ UMAP complete")

## Metric 1: Linear Accessibility (R²)

**Question:** Can we predict carry count from latent space using a linear model?

**Method:** Train Ridge regression to predict carry count, measure R².

**Interpretation:**
- R² ≈ 1: Perfect linear accessibility
- R² ≈ 0: No linear relationship

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    latent_space, carry_counts, test_size=0.2, random_state=42
)

# Train linear probe
probe = Ridge(alpha=1.0)
probe.fit(X_train, y_train)

# Evaluate
y_pred_train = probe.predict(X_train)
y_pred_test = probe.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
corr_test, _ = pearsonr(y_test, y_pred_test)

print("="*60)
print("LINEAR ACCESSIBILITY ANALYSIS")
print("="*60)
print(f"\nTrain R²: {r2_train:.4f}")
print(f"Test R²:  {r2_test:.4f}")
print(f"Test Correlation: {corr_test:.4f}")
print(f"\nInterpretation: {r2_test:.1%} of carry count variance")
print(f"is explained by a LINEAR model of the latent space.")
print("="*60)

In [ ]:
# Visualize predictions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
ax1.scatter(y_test, y_pred_test, alpha=0.6, s=50)
ax1.plot([0, 4], [0, 4], 'r--', linewidth=2, label='Perfect prediction')
ax1.set_xlabel('True Carry Count', fontsize=12)
ax1.set_ylabel('Predicted Carry Count', fontsize=12)
ax1.set_title(f'Linear Probe Performance (R² = {r2_test:.4f})', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Residuals
residuals = y_test - y_pred_test
ax2.scatter(y_pred_test, residuals, alpha=0.6, s=50)
ax2.axhline(0, color='r', linestyle='--', linewidth=2)
ax2.set_xlabel('Predicted Carry Count', fontsize=12)
ax2.set_ylabel('Residual', fontsize=12)
ax2.set_title('Residual Plot', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'linear_probe_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

## Metric 2: Geometric Clustering (Silhouette)

**Question:** Are points with similar carry counts clustered together in space?

**Method:** Compute Silhouette score using carry count as cluster labels.

**Interpretation:**
- Silhouette ≈ 1: Well-separated clusters
- Silhouette ≈ 0: Overlapping, continuous distribution
- Silhouette < 0: Points assigned to wrong clusters

In [ ]:
# Compute silhouette scores
sil_carry = silhouette_score(latent_space, carry_counts)
sil_bit = silhouette_score(latent_space, bit_positions)

print("="*60)
print("GEOMETRIC CLUSTERING ANALYSIS")
print("="*60)
print(f"\nSilhouette (carry count):    {sil_carry:.4f}")
print(f"Silhouette (bit position):   {sil_bit:.4f}")
print(f"\nInterpretation:")
if sil_carry > 0.5:
    print("  Strong clustering - discrete semantic groups")
elif sil_carry > 0.3:
    print("  Moderate clustering - some separation")
else:
    print("  Weak clustering - continuous distribution")
print("="*60)

In [ ]:
# Visualize clustering quality
from sklearn.metrics import silhouette_samples

# Compute per-sample silhouette scores
sample_silhouette_values = silhouette_samples(latent_space, carry_counts)

fig, ax = plt.subplots(figsize=(10, 6))

y_lower = 10
for i in range(5):  # 0-4 carry counts
    # Get silhouette scores for this cluster
    ith_cluster_silhouette_values = sample_silhouette_values[carry_counts == i]
    ith_cluster_silhouette_values.sort()
    
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = plt.cm.viridis(float(i) / 5)
    ax.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values,
                      facecolor=color, edgecolor=color, alpha=0.7)
    
    # Label the silhouette plots with their cluster numbers at the middle
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    
    y_lower = y_upper + 10

ax.set_title('Silhouette Analysis by Carry Count', fontweight='bold')
ax.set_xlabel('Silhouette Coefficient')
ax.set_ylabel('Carry Count')
ax.axvline(x=sil_carry, color="red", linestyle="--", label=f'Average: {sil_carry:.3f}')
ax.axvline(x=0, color="black", linestyle="-", linewidth=0.5)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'silhouette_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

## The Paradox Visualization

**High R² + Low Silhouette = Continuous Semantic Gradients**

This is the core finding of the paper.

In [ ]:
# Create paradox visualization
fig, ax = plt.subplots(figsize=(10, 6))

metrics = ['Linear\nAccessibility\n(R²)', 'Geometric\nClustering\n(Silhouette)']
values = [r2_test, sil_carry]
colors = ['#2ecc71' if v > 0.5 else '#e74c3c' for v in values]

bars = ax.barh(metrics, values, color=colors, alpha=0.7, height=0.6)
ax.set_xlim(0, 1)
ax.set_xlabel('Score', fontsize=13)
ax.set_title('The Accessibility-Clustering Paradox', fontsize=15, fontweight='bold')
ax.axvline(0.5, color='gray', linestyle='--', alpha=0.5, linewidth=2, label='Threshold (0.5)')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, values)):
    ax.text(val + 0.02, i, f'{val:.4f}', va='center', fontweight='bold', fontsize=12)

# Add interpretation box
textstr = '\n'.join([
    'HIGH Linear Accessibility:',
    '  → Semantic info is linearly decodable',
    '',
    'LOW Geometric Clustering:',
    '  → NOT organized into discrete clusters',
    '',
    'Conclusion: Continuous semantic gradients',
])
props = dict(boxstyle='round', facecolor='wheat', alpha=0.3)
ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', horizontalalignment='right', bbox=props)

ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'accessibility_clustering_paradox.png'), dpi=200, bbox_inches='tight')
plt.show()

print("\n" + "="*70)
print("THE PARADOX")
print("="*70)
print(f"Linear Accessibility (R²): {r2_test:.4f} → HIGH")
print(f"Geometric Clustering (Sil): {sil_carry:.4f} → LOW")
print("\nSemantic information is LINEARLY ACCESSIBLE")
print("without requiring GEOMETRIC CLUSTERING.")
print("="*70)

## Latent Space Visualizations

Visualize the latent space colored by different semantic properties.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. PCA - Carry count
ax = axes[0, 0]
scatter = ax.scatter(latent_pca[:, 0], latent_pca[:, 1], 
                     c=carry_counts, cmap='viridis', s=30, alpha=0.6)
ax.set_title('PCA: Carry Count', fontsize=13, fontweight='bold')
ax.set_xlabel(f'PC1 ({explained_var[0]:.1%})')
ax.set_ylabel(f'PC2 ({explained_var[1]:.1%})')
plt.colorbar(scatter, ax=ax, label='Carry Count')
ax.grid(True, alpha=0.3)

# 2. PCA - Bit position
ax = axes[0, 1]
scatter = ax.scatter(latent_pca[:, 0], latent_pca[:, 1], 
                     c=bit_positions, cmap='plasma', s=30, alpha=0.6)
ax.set_title('PCA: Bit Position', fontsize=13, fontweight='bold')
ax.set_xlabel(f'PC1 ({explained_var[0]:.1%})')
ax.set_ylabel(f'PC2 ({explained_var[1]:.1%})')
plt.colorbar(scatter, ax=ax, label='Bit Position')
ax.grid(True, alpha=0.3)

# 3. PCA - Input sum (data not available)
ax = axes[0, 2]
ax.text(0.5, 0.5, 'Input data\nnot available', ha='center', va='center', fontsize=14, transform=ax.transAxes)
ax.set_title('PCA: Input Sum', fontsize=13, fontweight='bold')
ax.axis('off')

# 4. UMAP - Carry count
ax = axes[1, 0]
scatter = ax.scatter(latent_umap[:, 0], latent_umap[:, 1], 
                     c=carry_counts, cmap='viridis', s=30, alpha=0.6)
ax.set_title('UMAP: Carry Count', fontsize=13, fontweight='bold')
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
plt.colorbar(scatter, ax=ax, label='Carry Count')
ax.grid(True, alpha=0.3)

# 5. UMAP - Bit position
ax = axes[1, 1]
scatter = ax.scatter(latent_umap[:, 0], latent_umap[:, 1], 
                     c=bit_positions, cmap='plasma', s=30, alpha=0.6)
ax.set_title('UMAP: Bit Position', fontsize=13, fontweight='bold')
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
plt.colorbar(scatter, ax=ax, label='Bit Position')
ax.grid(True, alpha=0.3)

# 6. UMAP - Input sum (data not available)
ax = axes[1, 2]
ax.text(0.5, 0.5, 'Input data\nnot available', ha='center', va='center', fontsize=14, transform=ax.transAxes)
ax.set_title('UMAP: Input Sum', fontsize=13, fontweight='bold')
ax.axis('off')

plt.suptitle('Delta Observer Latent Space: Multiple Perspectives', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'latent_space_comprehensive.png'), dpi=200, bbox_inches='tight')
plt.show()

## Perturbation Stability Analysis

Test whether the latent space is robust to small perturbations in the input activations.

In [ ]:
# Simulate perturbations
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Load model
sys.path.append(models_dir)  # Add models directory to path for delta_observer.py
from delta_observer import DeltaObserver, DeltaObserverDataset

model = DeltaObserver(mono_dim=64, comp_dim=64, latent_dim=16)
model.load_state_dict(torch.load(os.path.join(models_dir, 'delta_observer_best.pt')))
model.eval()

dataset = DeltaObserverDataset(os.path.join(data_dir, 'delta_observer_dataset.npz'))

print("Testing perturbation stability...")

perturbation_distances = []
n_samples = 100

with torch.no_grad():
    for i in range(min(n_samples, len(dataset))):
        sample = dataset[i]
        mono_act = sample['mono_act'].unsqueeze(0)
        comp_act = sample['comp_act'].unsqueeze(0)
        
        # Original latent
        latent_orig = model.encode(mono_act, comp_act).numpy()
        
        # Perturb activations
        mono_perturbed = mono_act + torch.randn_like(mono_act) * 0.1
        comp_perturbed = comp_act + torch.randn_like(comp_act) * 0.1
        
        # Perturbed latent
        latent_perturbed = model.encode(mono_perturbed, comp_perturbed).numpy()
        
        # Measure distance
        dist = np.linalg.norm(latent_orig - latent_perturbed)
        perturbation_distances.append(dist)

mean_dist = np.mean(perturbation_distances)
std_dist = np.std(perturbation_distances)

print(f"\nMean perturbation distance: {mean_dist:.4f} ± {std_dist:.4f}")
print(f"Interpretation: {'Stable' if mean_dist < 1.0 else 'Unstable'} latent space")

# Visualize
plt.figure(figsize=(10, 5))
plt.hist(perturbation_distances, bins=30, alpha=0.7, edgecolor='black')
plt.axvline(mean_dist, color='r', linestyle='--', linewidth=2, label=f'Mean: {mean_dist:.3f}')
plt.xlabel('Perturbation Distance')
plt.ylabel('Frequency')
plt.title('Latent Space Stability Under Perturbation', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'perturbation_stability.png'), dpi=150, bbox_inches='tight')
plt.show()

## Summary Report

In [ ]:
print("\n" + "="*70)
print("DELTA OBSERVER: ANALYSIS SUMMARY")
print("="*70)

print("\n📊 DIMENSIONALITY")
print(f"  Latent space: 16D")
print(f"  PCA (2 components): {explained_var[:2].sum():.1%} variance")
print(f"  PCA (5 components): {explained_var[:5].sum():.1%} variance")

print("\n🎯 LINEAR ACCESSIBILITY (R²)")
print(f"  Train: {r2_train:.4f}")
print(f"  Test:  {r2_test:.4f}")
print(f"  → Semantic information is LINEARLY ACCESSIBLE")

print("\n📐 GEOMETRIC CLUSTERING (Silhouette)")
print(f"  Carry count: {sil_carry:.4f}")
print(f"  Bit position: {sil_bit:.4f}")
print(f"  → Minimal clustering, CONTINUOUS distribution")

print("\n🔬 PERTURBATION STABILITY")
print(f"  Mean distance: {mean_dist:.4f} ± {std_dist:.4f}")
print(f"  → {'Stable' if mean_dist < 1.0 else 'Unstable'} representations")

print("\n" + "="*70)
print("KEY FINDING: THE ACCESSIBILITY-CLUSTERING PARADOX")
print("="*70)
print("\nSemantic information can be LINEARLY ACCESSIBLE (R² ≈ 0.95)")
print("WITHOUT exhibiting GEOMETRIC CLUSTERING (Silhouette ≈ 0.03).")
print("\nThis challenges the assumption that interpretability requires")
print("discrete, spatially separated feature clusters.")
print("\nInstead, semantic primitives exist as CONTINUOUS GRADIENTS.")
print("="*70)

print("\n📁 Figures saved:")
print("  - pca_variance.png")
print("  - linear_probe_analysis.png")
print("  - silhouette_analysis.png")
print("  - accessibility_clustering_paradox.png")
print("  - latent_space_comprehensive.png")
print("  - perturbation_stability.png")

## Next Steps

For complete end-to-end reproduction, see **`99_full_reproduction.ipynb`**.